In [42]:
%%capture
!pip install asteroid
!git clone https://github.com/asteroid-team/asteroid.git
%cd asteroid/egs/wsj0-mix-var/Multi-Decoder-DPRNN

SyntaxError: invalid syntax (3750872618.py, line 1)

In [43]:
# prepare inputs
import torch, torchaudio
from IPython.display import display, Audio
file = "/media/sandy/Work2/dsp_task4/human-voices/4_mixture.wav"
mixture, sample_rate = torchaudio.load(file)
print("mixture audio(Input)")
display(Audio(data=mixture, rate=sample_rate))

RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\Users\Khalaf\AppData\Roaming\Python\Python313\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\Users\Khalaf\AppData\Roaming\Python\Python313\site-packages\torchcodec\libtorchcodec_core7.dll
FFmpeg version 6: Could not load this libr

In [39]:
# --------------------------------------------------------------
# 1. Imports & model loading
# --------------------------------------------------------------
from model import MultiDecoderDPRNN
import torch
import numpy as np
from IPython.display import display, Audio
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Allow the two PL callbacks that are stored in the checkpoint
torch.serialization.add_safe_globals([ModelCheckpoint, EarlyStopping])

# Load pretrained model
model = MultiDecoderDPRNN.from_pretrained("JunzheJosephZhu/MultiDecoderDPRNN").eval()

# --------------------------------------------------------------
# 2. Move model & mixture to GPU (if available)
# --------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ---->  mixture & sample_rate come from the previous cell  <----
#     (shape must be [1, T], dtype=torch.float32, sr=16000)
mixture = mixture.to(device)               # <-- move to GPU
print(f"Mixture shape: {mixture.shape}, sr: {sample_rate}")

# --------------------------------------------------------------
# 3. Separate the 4 sources
# --------------------------------------------------------------
with torch.no_grad():
    sources_est = model.separate(mixture).cpu()   # [4, T] or [4,1,T]

# Remove possible channel dimension
if sources_est.ndim == 3:
    sources_est = sources_est.squeeze(1)          # -> [4, T]

# --------------------------------------------------------------
# 4. Hard-coded gain values per source (change freely, 0-2)
# --------------------------------------------------------------
gains = [1.0, 1.0, 0.0, 0.0]      # <-- edit here
assert all(0 <= g <= 2 for g in gains), "Gains must be in [0, 2]!"

# --------------------------------------------------------------
# 5. Dominant-frequency helper (exactly your code)
# --------------------------------------------------------------
def get_dominant_frequency(waveform, sample_rate):
    if waveform.dim() == 2:
        waveform = waveform.mean(dim=0)          # mono
    waveform = waveform - waveform.mean()        # DC removal
    fft_vals = torch.fft.rfft(waveform)
    magnitude = torch.abs(fft_vals)
    freqs = torch.fft.rfftfreq(waveform.shape[-1], d=1.0 / sample_rate)
    return freqs[magnitude.argmax()].item()

# --------------------------------------------------------------
# 6. Process each source – original + after gain
# --------------------------------------------------------------
processed_sources = []

for i, source in enumerate(sources_est):
    src = source.squeeze()                     # make sure [T]

    # ---- original ----
    print(f"\nEstimated source {i} (original):")
    display(Audio(data=src.numpy(), rate=sample_rate))

    dom_freq = get_dominant_frequency(src, sample_rate)
    print(f" Dominant frequency: {dom_freq:.2f} Hz")

    # ---- apply gain ----
    gain = gains[i]
    processed = src * gain
    processed_sources.append(processed)

    print(f" Gain applied: {gain}")
    print(f"After applying gain:")
    display(Audio(data=processed.numpy(), rate=sample_rate))
    print("-" * 70)

# --------------------------------------------------------------
# 7. Sum all gained sources + normalize
# --------------------------------------------------------------
final_mix = torch.stack(processed_sources).sum(dim=0)
final_mix = final_mix / final_mix.abs().max()   # avoid clipping

print("\nFinal recombined audio after applying gain to each source:")
display(Audio(data=final_mix.numpy(), rate=sample_rate))

# --------------------------------------------------------------
# 8. (Optional) Save the result
# --------------------------------------------------------------
import soundfile as sf
sf.write("final_mix_with_gains.wav", final_mix.numpy(), sample_rate)
print("Saved: final_mix_with_gains.wav")

Mixture shape: torch.Size([1, 105600]), sr: 16000

Estimated source 0 (original):


 Dominant frequency: 403.33 Hz
 Gain applied: 1.0
After applying gain:


----------------------------------------------------------------------

Estimated source 1 (original):


 Dominant frequency: 477.42 Hz
 Gain applied: 1.0
After applying gain:


----------------------------------------------------------------------

Estimated source 2 (original):


 Dominant frequency: 207.58 Hz
 Gain applied: 0.0
After applying gain:


----------------------------------------------------------------------

Final recombined audio after applying gain to each source:


Saved: final_mix_with_gains.wav
